In [1]:
import pickle
import pandas as pd
import fasttext
import numpy as np
import os

# CSV dosyasını yükleme
print("CSV dosyası yükleniyor...")
df = pd.read_csv("data_all_news.csv")
print(f"CSV dosyası başarıyla yüklendi. Toplam satır sayısı: {len(df)}")

# FastText modelini yükleme
print("FastText modeli yükleniyor...")
fasttext.FastText.eprint = lambda x: None
model = fasttext.load_model('models/cc.tr.300.bin')  # Türkçe FastText modeli
print("FastText modeli başarıyla yüklendi.")

# Kelime vektörlerini önceden hesaplama
print("Modelden kelime vektörleri çekiliyor...")
word_vectors = {word: model.get_word_vector(word) for word in model.get_words()}
print(f"Toplam kelime vektörü sayısı: {len(word_vectors)}")

# Haber başlıklarını vektörleme
def fasttext_vectorize(text, word_vectors, dim=300):
    words = text.split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros(dim)

# Parça başına işleme ve kaydetme
batch_size = 100
output_file = "fasttext_vectors.pkl"

# Eğer çıktı dosyası varsa kaldığımız yerden devam etmek için yükleme
if os.path.exists(output_file):
    print("Mevcut çıktı dosyası bulunuyor. İşlem kaldığı yerden devam edecek...")
    with open(output_file, "rb") as file:
        existing_data = pickle.load(file)
        X, y = existing_data
        X, y = list(X), list(y)  # Listeye çevirip ekleme yapacağız
    processed_count = len(X)
    print(f"{processed_count} başlık daha önce işlenmiş.")
else:
    print("Yeni işlem başlatılıyor...")
    X, y = [], []
    processed_count = 0

# İşleme devam et
print("Başlıklar işleniyor...")
for i in range(processed_count, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    print(f"Batch işlemi başlıyor: {i} - {i + len(batch)}")
    batch_vectors = [fasttext_vectorize(title, word_vectors) for title in batch['Title']]
    X.extend(batch_vectors)
    y.extend(batch['Label'])
    print(f"Batch tamamlandı: {i} - {i + len(batch)}. Şimdi dosyaya kaydediliyor...")

    # Her batch sonrası dosyayı güncelleme
    with open(output_file, "wb") as file:
        pickle.dump((X, y), file)
    print(f"Batch dosyaya kaydedildi: {i} - {i + len(batch)}")

print("Tüm veriler başarıyla işlendi ve kaydedildi.")


CSV dosyası yükleniyor...
CSV dosyası başarıyla yüklendi. Toplam satır sayısı: 12723
FastText modeli yükleniyor...
FastText modeli başarıyla yüklendi.
Modelden kelime vektörleri çekiliyor...
Toplam kelime vektörü sayısı: 2000000
Yeni işlem başlatılıyor...
Başlıklar işleniyor...
Batch işlemi başlıyor: 0 - 100
Batch tamamlandı: 0 - 100. Şimdi dosyaya kaydediliyor...
Batch dosyaya kaydedildi: 0 - 100
Batch işlemi başlıyor: 100 - 200
Batch tamamlandı: 100 - 200. Şimdi dosyaya kaydediliyor...
Batch dosyaya kaydedildi: 100 - 200
Batch işlemi başlıyor: 200 - 300
Batch tamamlandı: 200 - 300. Şimdi dosyaya kaydediliyor...
Batch dosyaya kaydedildi: 200 - 300
Batch işlemi başlıyor: 300 - 400
Batch tamamlandı: 300 - 400. Şimdi dosyaya kaydediliyor...
Batch dosyaya kaydedildi: 300 - 400
Batch işlemi başlıyor: 400 - 500
Batch tamamlandı: 400 - 500. Şimdi dosyaya kaydediliyor...
Batch dosyaya kaydedildi: 400 - 500
Batch işlemi başlıyor: 500 - 600
Batch tamamlandı: 500 - 600. Şimdi dosyaya kaydediliyo

In [4]:
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

# PKL dosyasını yükleme
with open("fasttext_vectors.pkl", "rb") as file:
    X, y = pickle.load(file)

# NumPy dizisine dönüştürme
X = np.array(X)
y = np.array(y)

# Eğitim ve test veri setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Eğitim verisi boyutu:", X_train.shape)
print("Test verisi boyutu:", X_test.shape)


Eğitim verisi boyutu: (10178, 300)
Test verisi boyutu: (2545, 300)


In [2]:
import pickle
with open("fasttext_vectors.pkl", "rb") as file:
    data = pickle.load(file)
    print(type(data))
    print(data.keys() if isinstance(data, dict) else "Not a dict")

<class 'tuple'>
Not a dict


In [5]:
# Numpy modülünü ve diğer bağımlılıkları yeniden yükleyip dosyayı tekrar kontrol edeceğim.
import numpy as np
import pickle
import pandas as pd
import fasttext
import numpy as np
import os

# PKL dosyasını tekrar yükleme denemesi
with open("fasttext_vectors.pkl", "rb") as file:
    X, y = pickle.load(file)

# CSV dosyasını yükleme
df = pd.read_csv("data_all_news.csv")

# PKL ve CSV uyum kontrolü
pkl_vector_count = len(X)
csv_title_count = len(df)

pkl_vector_count, csv_title_count, X[:3], y[:3]  # İlk 3 vektör ve label kontrolü


(12723,
 12723,
 [array([-0.02357707,  0.01423337,  0.04190415,  0.0055367 , -0.04661968,
          0.02325811,  0.05232373,  0.02731863,  0.02573657, -0.0662409 ,
         -0.03778307, -0.04119961,  0.00113459, -0.02766742, -0.03796528,
         -0.05237185, -0.00388695,  0.02929966, -0.03030977,  0.09183662,
         -0.0159668 , -0.026929  ,  0.01291961, -0.00407556, -0.00947283,
         -0.01474358,  0.0389166 ,  0.00025762, -0.00288128, -0.03617932,
          0.01654119, -0.02822708,  0.01660349, -0.03996019, -0.00445063,
         -0.00616995,  0.02046519,  0.05188668, -0.02322857,  0.01258788,
          0.01305305, -0.05498641, -0.01114177, -0.004791  , -0.00055074,
         -0.0312843 ,  0.006508  , -0.00183747,  0.01968789, -0.02317334,
          0.03091307, -0.03787336, -0.0518472 ,  0.01288313, -0.01981827,
         -0.00034296,  0.0175668 , -0.02166904,  0.03164417,  0.01046822,
          0.0112676 ,  0.02424164, -0.04169977,  0.0045274 ,  0.01876269,
         -0.02162689, 

In [9]:
index = 5  # Rastgele bir index seçip kontrol ettim pkl dosyasını
print("Başlık:", df.iloc[index]['Title'])
print("Vektör:", X[index])
print("Etiket:", y[index])

print("--------Vektörlerin boyutu---------") #Vektörlerin boyutu 300 mü ona baktım
print(all([len(vec) == 300 for vec in X]))

Başlık: çanakkalede feribot seferleri iptal edildi
Vektör: [ 2.41813459e-03  1.63926631e-02  6.58961758e-02 -1.25431390e-02
 -1.27056511e-02 -1.75226517e-02  9.51198954e-03 -7.69971823e-03
  1.36706177e-02 -2.19522379e-02  1.65014286e-02 -6.84269890e-03
  1.54203000e-02 -4.40616012e-02 -4.28600088e-02 -2.20406167e-02
 -5.98925352e-02  1.92406252e-02  2.97854394e-02  2.52903290e-02
  1.52818789e-03  7.80502567e-03  2.73921695e-02  5.89762330e-02
 -4.97735590e-02 -1.61523279e-02 -2.15278529e-02 -1.68913789e-03
  2.06501745e-02  1.60810295e-02 -1.56401973e-02 -1.57628581e-02
 -4.76043522e-02  5.52010257e-03  1.16446111e-02  2.23397911e-02
  2.45769713e-02  3.46132144e-02  3.00561631e-04 -1.07774828e-02
 -2.86762714e-02 -2.59723477e-02 -4.65515032e-02 -2.17412300e-02
 -6.19365163e-02 -2.99929529e-02  6.72706868e-03 -5.85382758e-03
 -6.67261407e-02  4.32605203e-03  1.55034121e-02 -2.01827642e-02
  2.23815013e-02 -1.06400661e-02  1.60786416e-02  4.20256406e-02
  3.46055105e-02  1.82574168e-0